In [11]:
from ssfAPI import *
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import glob
import json

In [12]:
def read_json(filepath):
    data = []
    with open(filepath, encoding="utf-8") as f:
        for idx_, row in enumerate(f):
            data_t = json.loads(row)
            sent = " ".join(data_t['words']).strip()
            data.append(sent)
    return data


In [13]:
urdu_train_sents = read_json("final_jsons/train_all.json")
urdu_val_sents = read_json("final_jsons/val_all.json")
urdu_test_sents = read_json("final_jsons/test_urdu.json")

In [25]:
chunks_train = []
chunks_test = []
chunks_val = []
c = 0
for folder in glob.glob('../Data/Urdu/*_with_ChunkHeads'):

    fileList = folderWalk(folder)

    # Filtering files
    newFileList = []
    for fileName in fileList :
        xFileName = fileName.split('/')[-1]
        if xFileName == 'err.txt' or xFileName.split('.')[-1] in ['comments','bak'] or xFileName[:4] == 'task' :
            continue
        else :
            newFileList.append(fileName)

    # Extracting chunks
    for fileName in newFileList :
        print(fileName)
        d = Document(fileName)
        for tree in d.nodeList : 
            try:
                train, test, val = False, False, False
                predicate = None
                consider_this_sentence = False
                for chunk in tree.nodeList:
                    temp = {}
                    if 'pbrel' in chunk.attributes:
                        consider_this_sentence = True
                    if  'NULL' not in chunk.name and 'drel' not in chunk.attributes:
                        predicate = chunk.attributes['head']
                if consider_this_sentence and predicate!= None:
                    words_t = []
                    for chunk in tree.nodeList:
                        for word_node in chunk.nodeList:
                                word = word_node.lex
                                if word != 'NULL':
                                    words_t.append(word)
                    sentence = " ".join(words_t)
                    sentence = re.sub(r'\s+', ' ', sentence)
                    if sentence in urdu_train_sents:
                        train = True
                    elif sentence in urdu_val_sents:
                        val = True
                    elif sentence in urdu_test_sents:
                        test = True
                    else:
                        # print(sentence)
                        c+=1
                    
                    for chunk in tree.nodeList:
                            if 'NULL' not in chunk.name and 'drel' in chunk.attributes:
                                if 'pbrel' in chunk.attributes:
                                    is_arg = True
                                    role_label = chunk.attributes['pbrel'].split(":")[0]
                                else:
                                    is_arg = False
                                    role_label = 0
#                                     predicate_name = 0
                                dependency = chunk.attributes['drel'].split(":")[0]
                                head_word_root = chunk.attributes['head']

                                
                                if 'af' in chunk.attributes:
                                    head_word_pos = chunk.attributes['af'].split(",")[1]
                                else:
                                    head_word_pos = chunk.type
                                    
                               

                                temp = {'chunk_type':chunk.type, 'predicate':predicate, 'is_arg':is_arg, 'role_label':role_label, 
                                        'head_word_root':head_word_root, 'head_word_pos':head_word_pos,
                                       'dependency':dependency}
                                for key in temp:
                                    if temp[key]=='' or temp[key] == None:
                                        temp[key]='Not Found'
                                
                                if train:
                                    chunks_train.append(temp)
                                elif val:
                                    chunks_val.append(temp)
                                elif test:
                                    chunks_test.append(temp)
            except Exception as e:
                print(e)
                print(chunk.attributes)
                continue
#                 break

../Data/Urdu/ComplexPredicates_with_ChunkHeads/fullnews_id_199848_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/ComplexPredicates_with_ChunkHeads/fullnews_id_199922_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/ComplexPredicates_with_ChunkHeads/fullnews_id_199815_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/ComplexPredicates_with_ChunkHeads/fullnews_id_199945_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/ComplexPredicates_with_ChunkHeads/fullnews_id_200190_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/ComplexPredicates_with_ChunkHeads/fullnews_id_200312_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/ComplexPredicates_with_ChunkHeads/fullnews_id_199941_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.pru

../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199939_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199974_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199938_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200115_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200393_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199899_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199946_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head

../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200341_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200162_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199959_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200332_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199865_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200385_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200271_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
..

../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200119_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200334_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200020_date_07_25_2009.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200295_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199936_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200046_date_07_25_2009.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200410_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/Simple

../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200152_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199991_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200172_date_07_25_2009.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200149_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200127_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199825_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200388_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head


../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200129_date_07_25_2009.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199995_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200189_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200418_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200150_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199914_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200316_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/Simpl

../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200193_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200258_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200261_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200035_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199852_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200211_date_07_25_2009.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200323_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.pb.he

../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200306_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200218_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200300_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200266_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200049_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199897_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199808_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn

../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199993_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200244_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200219_date_07_25_2009.utf8.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200208_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_199873_date_07_25_2009.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200228_date_07_25_2009.cml.V.tkn.cml_updated.mo.pos.chnk.prun.posn.pb.head
../Data/Urdu/SimplePredicates_with_ChunkHeads/fullnews_id_200157_date_07_25_2009.txt.utf8.cml.V.tkn.cml_updated.wx.mo.utf.pos.chnk.prun.posn.pb.head


In [27]:
df_train = pd.DataFrame(chunks_train).dropna()
df_train.to_csv('urdu_data_train.csv', index=False)

df_val = pd.DataFrame(chunks_val).dropna()
df_val.to_csv('urdu_data_val.csv', index=False)

df_test = pd.DataFrame(chunks_test).dropna()
df_test.to_csv('urdu_data_test.csv', index=False)


In [29]:
df_train.head()

,chunk_type,predicate,is_arg,role_label,head_word_root,head_word_pos,dependency
0,NP,اور,True,ARG2-LOC,لیبیا,n,k7
1,NP,اور,True,ARGM-PRX,پابندیاں,n,pof
2,VGF,اور,False,0,لگائی,v,ccof
3,NP,اور,True,ARGM-PRX,کارروائی,n,pof
4,VGF,اور,False,0,کی,v,ccof
